In [76]:
import requests
import pandas as pd
from pprint import pprint
import time
import json
import csv
import math
import numpy

In [77]:
url="https://api.census.gov/data/2018/pep/population?get=DENSITY,POP,GEONAME,PLACE,STATE&for=PLACE:*&DATE_CODE=11&key=2e2f28e7afbd90fe2bc31ba0efc117439d6264d9"
data = requests.get(url).json()

In [78]:
df=pd.DataFrame(data=data[1:],
                columns=data[0])

In [79]:

df[['City','State']] = df['GEONAME'].str.split(', ',expand=True)
df['City']=df['City'].str.replace(' city*', '')
df['City']=df['City'].str.replace(' town*', '')
df['City']=df['City'].str.replace(' borough*', '')
df['City']=df['City'].str.replace(' village*', '')
df['City']=df['City'].str.replace(' municipality*', '')
df['City']=df['City'].str.replace(' \(.*', '')
df['City']=df['City'].str.replace('-*government*', '')
df['City']=df['City'].str.replace('Macon-Bibb County', 'Macon')
df['City']=df['City'].str.replace('Augusta-Richmond County consolidated*', 'Augusta')
df['City']=df['City'].str.replace('Lexington-Fayette urban county*', 'Lexington')
df['City']=df['City'].str.replace('Nashville-Davidson metropolitan*', 'Nashville')
df['City']=df['City'].str.replace('Louisville/Jefferson County metro*', 'Louisville')
df['City']=df['City'].str.replace('Athens-Clarke County unified*', 'Athens')

df2 = df.drop(columns=["GEONAME", "PLACE",'STATE','DATE_CODE','state','place'])

df2['POP']=pd.to_numeric(df2['POP'])
df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19495 entries, 0 to 19494
Data columns (total 4 columns):
DENSITY    19493 non-null object
POP        19495 non-null int64
City       19495 non-null object
State      19495 non-null object
dtypes: int64(1), object(3)
memory usage: 609.3+ KB


,DENSITY,POP,City,State
0,164.90074649,2563,Abbeville,Alabama
1,171.82307679,4325,Adamsville,Alabama
2,192.25827749,724,Addison,Alabama
3,481.20360575,330,Akron,Alabama
4,1320.9089944,33340,Alabaster,Alabama


In [80]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
states=pd.DataFrame.from_dict(us_state_abbrev,orient='index', columns=['abrev'])
states.reset_index(level=0, inplace=True)
states=states.rename(columns={states.columns[0]:'name'})
states.head()

,name,abrev
0,Mississippi,MS
1,Oklahoma,OK
2,Delaware,DE
3,Minnesota,MN
4,Illinois,IL


In [85]:
df1 = pd.merge(df2, states,  how='left', left_on=['State'], right_on = ['name'])
df1 = df1.drop(columns=['name'])

df1=df1.sort_values(by='City', ascending=False)

In [86]:
df1.to_csv("popden.csv", encoding="utf-8" )